# Classification: Bayesian Search

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [1]:
#to find the best set of parameter setting, we can run a grid search
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import random
import numpy as np

from sklearn import tree
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import wittgenstein as lw
import keras_tuner
import keras
from keras_tuner import HyperParameters
import tensorflow as tf

import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from statistics import mean, stdev
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from scipy.stats import loguniform as sp_loguniform
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score


2024-12-18 12:04:15.766332: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-18 12:04:15.784236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734519855.803455   58573 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734519855.808808   58573 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 12:04:15.828386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [12]:
# Merge results from previous random search
USER_1 = 'Jacopo'
USER_2 = 'Simone'

models = ['ada_boost', 'nn', 'xgb', 'naive_bayes', 'random_forest', 'decision_tree', 'svm', 'rule_based', 'knn']

for model in models:
    path_1 = f'../../data/ml_datasets/undersampling/model_selection/{USER_1}_{model}_results.csv'
    path_2 = f'../../data/ml_datasets/undersampling/model_selection/{USER_2}_{model}_results.csv'

    concatenate_path = f'../../data/ml_datasets/undersampling/model_selection/{model}_results.csv'

    df1 = pd.read_csv(path_1)
    df2 = pd.read_csv(path_2)

    # Concatena le righe
    df_concatenato = pd.concat([df1, df2], ignore_index=True)

    # Salva il risultato in un nuovo CSV
    df_concatenato.to_csv(concatenate_path, index=False)  #to concatenate the two files

In [2]:
columns_to_see = ['mean_test_f1_micro', 'std_test_f1_micro', 'mean_test_f1_1', 'std_test_f1_1', 'mean_test_f1_0', 'std_test_f1_0']

In [11]:
# df = pd.read_csv('../../data/ml_datasets/undersampling/model_selection/ada_boost_results.csv')
# df.sort_values(by='mean_test_f1_micro', ascending=False, inplace=True)
# params= [col for col in df.columns if col.startswith("param_classifier__")]
# df.head(n=10)[columns_to_see+params]

In [3]:
models = ['ada_boost', 'nn', 'xgb', 'naive_bayes', 'random_forest', 'decision_tree', 'svm', 'rule_based', 'knn']

df_results = pd.read_csv('../../data/ml_datasets/undersampling/model_selection/nn_results.csv')
df_results = df_results.rename(columns={'mean_f1_micro': 'mean_test_f1_micro', 
                                        'std_f1_micro': 'std_test_f1_micro',
                                        'mean_f1_1': 'mean_test_f1_1',
                                        'std_f1_1': 'std_test_f1_1',
                                        'mean_f1_0': 'mean_test_f1_0',
                                        'std_f1_0': 'std_test_f1_0'})
print(df_results.columns)
df_results = df_results[columns_to_see]
df_results['model'] = 'nn'
models.remove('nn')

columns_to_see = ['model'] + columns_to_see
for model in models:
    path = f'../../data/ml_datasets/undersampling/model_selection/{model}_results.csv'

    df = pd.read_csv(path)
    df['model'] = model
    df['mean_test_f1_macro'] = (df['mean_test_f1_1'] + df['mean_test_f1_0']) / 2
    df.sort_values(by='mean_test_f1_macro', ascending=False, inplace=True)
    df = df.head(10)
    df = df[columns_to_see + ['mean_test_f1_macro']]

    df_results = pd.concat([df_results, df], axis=0)

Index(['batch_size', 'epochs', 'units_layer1', 'units_layer2', 'drop_rate',
       'learning_rate', 'mean_test_f1_micro', 'std_test_f1_micro',
       'mean_test_f1_0', 'std_test_f1_0', 'mean_test_f1_1', 'std_test_f1_1'],
      dtype='object')


In [7]:
df_results.sort_values(by='mean_test_f1_macro', ascending=False, inplace=True)
df_results.head(40)

,mean_test_f1_micro,std_test_f1_micro,mean_test_f1_1,std_test_f1_1,mean_test_f1_0,std_test_f1_0,model,mean_test_f1_macro
0,0.725322,0.002026,0.419911,0.003151,0.820059,0.001453,random_forest,0.619985
50,0.724990,0.002114,0.419359,0.003605,0.819827,0.001481,random_forest,0.619593
51,0.722791,0.002282,0.417092,0.003846,0.818156,0.001593,random_forest,0.617624
1,0.722686,0.002104,0.417035,0.003778,0.818071,0.001451,random_forest,0.617553
52,0.722207,0.002004,0.415785,0.003599,0.817781,0.001380,random_forest,0.616783
2,0.722276,0.002406,0.415650,0.003703,0.817853,0.001732,random_forest,0.616752
53,0.721899,0.001767,0.414652,0.003378,0.817626,0.001220,random_forest,0.616139
3,0.719406,0.001844,0.411937,0.003243,0.815744,0.001277,random_forest,0.613841
4,0.718787,0.001393,0.411841,0.002804,0.815219,0.000973,random_forest,0.613530
56,0.718341,0.001925,0.411940,0.003375,0.814824,0.001332,random_forest,0.613382


Winner models for UnderSampling:
- Random Forests
- XGB
- Decision Tree
- Rule-Based